In [77]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import random
import pandas as pd
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

[nltk_data] Downloading package stopwords to /Users/ylii/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ylii/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [78]:
import sqlite3 ,csv
conn = sqlite3.connect('db_sentiment_v2.sqlite3')
print("Opened.")
cur = conn.cursor()

Opened.


In [79]:
#test database
cur = conn.execute("SELECT tweet_text from psa_tweet LIMIT 5")
result = cur.fetchall()

for x in result:
  print(x)

('I enjoyed speaking with @actuary_dot_org about my bill to create a new service program for caregiving #CARECorps http://t.co/irs0UIiM8L',)
('We are concerned &amp; actively seeking answers on reports of increased enforcement activity by ICE in our communities https://t.co/WXEbQju40q',)
('RT @RepLarryBucshon: Proud to join @SpeakerRyan for this! Tune in here: https://t.co/3FFrvDHzv1. https://t.co/rVsq5wImZ8',)
("WSJ: likely that 'tens of millions' of records were stolen #AnthemHack http://t.co/m6t6uiDmyx",)
('"The upshot is one less Democrat in Washington for Missouri." MO Republicans say: "Thanks, @russcarnahan!" http://t.co/FxEPpHY0 #comeonruss',)


In [80]:
#create a dataframe with all tweets 
df = pd.read_sql_query("SELECT tweet_text from psa_tweet;", conn)
df

,tweet_text
0,I enjoyed speaking with @actuary_dot_org about...
1,We are concerned &amp; actively seeking answer...
2,RT @RepLarryBucshon: Proud to join @SpeakerRya...
3,WSJ: likely that 'tens of millions' of records...
4,"""The upshot is one less Democrat in Washington..."
5,Proverbs 4:23 Watch over your heart with all ...
6,Competitive fed grant to help replace Champ Cl...
7,"RT @RepTomGraves: On this day in 1788, the #Co..."
8,Today's #: $54M. HHS Sec. Sebelius has $54M to...
9,RT @ThisWeekABC: Will the GOP's platform chang...


In [81]:
#save alltweets to a text file
df.to_csv('tweets.txt', index=False, sep=' ', header=None)

In [82]:
#text cleaning
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [83]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /Users/ylii/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [84]:
#filter stopwords
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /Users/ylii/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [85]:
#prepare text for topic modeling
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [86]:
text_data = []
with open('tweets.txt') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

['weekinreview', 'startupact', 'watch']
['laredo', 'tercer', 'puerto', 'dinamico', 'manana', 'nuevo', 'laredo']
['simple', 'fair', '4jobs', 'agree', 'photo']
['harness', 'technology', 'innovation']
['equal', 'equal', 'access', 'healthcare', 'womensequalityday', 'remind']
['story', '@actingsecharris', 'hear', 'worker', 'today', 'president', 'proposal', 'mwraise', 'important']
['celebrate', 'roshhashanah', 'happy', 'sweet']
['@repandrecarson', 'million', 'hardworking', 'american', 'benefit', '@housegop', 'join', 'raisethewage']
['president', 'elect', 'demonstrate', 'using', 'power', 'presidency', 'enhance', 'business']
['excite', 'auburn', 'homecoming', 'talking', '@auburnimgsports']
['gathering', 'morning', 'meeting', 'marshall', 'county', 'industrial', 'development', 'authority']
['think', 'congress', 'fulfill', 'constitutional', 'budget', 'nobudgetnopay', 'ohgop']
['@randstudents', '.@randpaul', 'check', 'reaction', 'iowa10']
['congratulations', 'brent', 'powell', 'swear', 'district',

In [87]:
#LDA WITH GENSIM
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(corpus, num_topics=10, id2word = dictionary, passes=30)

In [88]:
#print 10 topics
topics = ldamodel.print_topics(num_topics=10, num_words=4)
for topic in topics:
    print(topic)

(0, '0.021*"senate" + 0.021*"@stltoday" + 0.021*"trillion" + 0.021*"mccaskill"')
(1, '0.020*"talking" + 0.020*"homecoming" + 0.020*"regard" + 0.020*"program"')
(2, '0.037*"meeting" + 0.037*"davis" + 0.019*"conduct" + 0.019*"mckinley"')
(3, '0.037*"business" + 0.019*"watch" + 0.019*"prevent" + 0.019*"american"')
(4, '0.023*"morning" + 0.023*"family" + 0.023*"congress" + 0.023*"veteran"')
(5, '0.045*"today" + 0.024*"visit" + 0.024*"group" + 0.024*"@americancancer"')
(6, '0.038*"together" + 0.038*"equal" + 0.038*"remind" + 0.038*"access"')
(7, '0.041*"today" + 0.021*"president" + 0.021*"@actingsecharris" + 0.021*"mwraise"')
(8, '0.019*"benefit" + 0.019*"action" + 0.019*"@repandrecarson" + 0.019*"hardworking"')
(9, '0.053*"laredo" + 0.028*"building" + 0.028*"great" + 0.028*"colorado"')
